In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kannada-tulu-train/tu_tr.txt
/kaggle/input/kannada-tulu-train/en_tr.txt
/kaggle/input/kannada-tulu-train/kn_tr.txt
/kaggle/input/kannada-tulu-test/en_dev.txt
/kaggle/input/kannada-tulu-test/tu_dev.txt
/kaggle/input/kannada-tulu-test/kn_dev.txt


In [2]:
!pip install torch torchvision torchaudio

In [3]:
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
#!git checkout v0.10.2
##!python -m pip install --user ./
!pip install --editable .
%cd ..

Cloning into 'fairseq'...
remote: Enumerating objects: 35203, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 35203 (delta 66), reused 80 (delta 48), pack-reused 35083
Receiving objects: 100% (35203/35203), 25.23 MiB | 31.69 MiB/s, done.
Resolving deltas: 100% (25556/25556), done.
/kaggle/working/fairseq
Obtaining file:///kaggle/working/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [4]:
!nvidia-smi

Thu Jul  4 08:34:01 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             27W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
!pip install git+https://github.com/vmujadia/tokenizer.git --upgrade

  Cloning https://github.com/vmujadia/tokenizer.git to /tmp/pip-req-build-lmd62ztt
  Running command git clone --filter=blob:none --quiet https://github.com/vmujadia/tokenizer.git /tmp/pip-req-build-lmd62ztt
  Resolved https://github.com/vmujadia/tokenizer.git to commit 93cd09b81702108a51c08c9796fd1cc941a1b98b
  Preparing metadata (setup.py) ... done
  Created wheel for IL-Tokenizer: filename=IL_Tokenizer-0.0.2-py3-none-any.whl size=7225 sha256=cc765a9a8f49066bd0ce0ad9170db01b1bed97bf9e956fa9a78bdd0376aab8ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-_zv708ew/wheels/9a/fb/5b/3d75bfde8561726121c09f0f0a83389c05312df8a513808c41
Successfully built IL-Tokenizer
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at h

In [6]:
!git clone https://github.com/moses-smt/mosesdecoder.git

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148459, done.
remote: Counting objects: 100% (887/887), done.
remote: Compressing objects: 100% (366/366), done.
remote: Total 148459 (delta 565), reused 797 (delta 517), pack-reused 147572
Receiving objects: 100% (148459/148459), 129.99 MiB | 24.88 MiB/s, done.
Resolving deltas: 100% (114591/114591), done.


In [7]:
!git clone https://github.com/rsennrich/subword-nmt.git

Cloning into 'subword-nmt'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 600 (delta 10), reused 12 (delta 4), pack-reused 576
Receiving objects: 100% (600/600), 253.58 KiB | 6.85 MiB/s, done.
Resolving deltas: 100% (359/359), done.


In [8]:
!ls mosesdecoder/scripts/training/clean-corpus-n.perl

mosesdecoder/scripts/training/clean-corpus-n.perl


In [9]:
# Use the datasets already downloaded in the Kaggle notebook
!cp /kaggle/input/kannada-tulu-train/en_tr.txt train.src
!cp /kaggle/input/kannada-tulu-train/tu_tr.txt train.tgt
!cp /kaggle/input/kannada-tulu-test/en_dev.txt valid.src
!cp /kaggle/input/kannada-tulu-test/tu_dev.txt valid.tgt


In [10]:
print ('Data Stats')
! wc -l train.*
! wc -l valid.*

Data Stats
   8300 train.src
   8300 train.tgt
  16600 total
  1000 valid.src
  1000 valid.tgt
  2000 total


In [11]:
from ilstokenizer import tokenizer
import codecs

def to_tokenize_and_lower(input_path, output_path):
  outfile = open(output_path, 'w')
  for line in codecs.open(input_path):
    line = line.strip()
    line = tokenizer.tokenize(line).lower()
    #print (line)
    outfile.write(line+'\n')
  outfile.close()

In [12]:
to_tokenize_and_lower('train.src','train.src.tkn')
to_tokenize_and_lower('train.tgt','train.tgt.tkn')

to_tokenize_and_lower('valid.src','valid.src.tkn')
to_tokenize_and_lower('valid.tgt','valid.tgt.tkn')

In [13]:
! cat train.src.tkn > train.all.tkn
! cat train.tgt.tkn >> train.all.tkn

In [14]:
! perl mosesdecoder/scripts/training/clean-corpus-n.perl -ratio 5 train src.tkn tgt.tkn train_filtered 1 512

clean-corpus.perl: processing train.src.tkn & .tgt.tkn to train_filtered, cutoff 1-512, ratio 5

Input sentences: 8300  Output sentences:  8290


In [15]:
print ('Data Stats')
! wc -l train*
! wc -l valid*

Data Stats
  16600 train.all.tkn
   8300 train.src
   8300 train.src.tkn
   8300 train.tgt
   8300 train.tgt.tkn
   8290 train_filtered.src.tkn
   8290 train_filtered.tgt.tkn
  66380 total
  1000 valid.src
  1000 valid.src.tkn
  1000 valid.tgt
  1000 valid.tgt.tkn
  4000 total


In [16]:
!python subword-nmt/subword_nmt/learn_bpe.py -s 3500 < train.all.tkn > train.codes

100%|######################################| 3500/3500 [00:07<00:00, 443.98it/s]


In [17]:
! head -n 10 train.codes

#version: 0.2
t h
th e</w>
i n
ಲ ್
a n
ರ ್
i s</w>
ಂ ದ
ತ ್


In [18]:
!python subword-nmt/subword_nmt/apply_bpe.py -c train.codes < train.src.tkn > train.en
!python subword-nmt/subword_nmt/apply_bpe.py -c train.codes < train.tgt.tkn > train.tu

!python subword-nmt/subword_nmt/apply_bpe.py -c train.codes < valid.src.tkn > valid.en
!python subword-nmt/subword_nmt/apply_bpe.py -c train.codes < valid.tgt.tkn > valid.tu

In [19]:
! head -n 10 train.en

the chang@@ e was writ@@ ten by ch@@ and@@ u
god gives everything when he gives it
also , the p@@ ain of the mind does not go away .
the pot@@ ential is on the mo@@ ve
there was a s@@ ci@@ ence that we did not under@@ stand e@@ as@@ ily .
in the inv@@ est@@ ig@@ ation of this case , as per the direc@@ tion of d@@ k district su@@ per@@ int@@ end@@ ent of police sh@@ ri h@@ ri@@ shi@@ k@@ esh s@@ on@@ a@@ wan@@ e i@@ ps , under the guid@@ ance of d@@ k district ad@@ d@@ it@@ i@@ on@@ al su@@ per@@ int@@ end@@ ent of police sh@@ ri v@@ .@@ b . bh@@ as@@ kar , ban@@ tw@@ al su@@ b - div@@ is@@ i@@ on@@ al de@@ pu@@ ty su@@ per@@ int@@ end@@ ent of police sh@@ ri val@@ ent@@ in d ' s@@ ou@@ z@@ a , ban@@ tw@@ al city police st@@ ation in@@ spe@@ ct@@ or ch@@ el@@ u@@ var@@ aj@@ u b . ban@@ tw@@ al city police st@@ ation su@@ b - in@@ spe@@ ct@@ or and st@@ af@@ f , ban@@ tw@@ al ru@@ ral police st@@ ation su@@ b - in@@ spe@@ ct@@ or and st@@ af@@ f , ban@@ tw@@ al city police st@@ ation cri

In [20]:
! head -n 10 train.tu

ಬದಲಾ@@ ವ@@ ಣೆ ಬರೆ@@ ಯಿ@@ ನಾರ್ ಚ@@ ಂದು ಪನ್ಪಿ@@ ನಾರ್
ದೇವೆರ್ ಕೊರ್@@ ನಗ ಮಾ@@ ತ@@ ಲ ಕೊರ್ಪೆರ್
ಅಂಚನೆ ಮನ@@ ಸು@@ ಗು ಆಯಿನ ಬೇನೆ ಆ ಬೇನೆ ಮಾ@@ ಜು@@ ಜಿ
ವಿಭವೆ ನಡತೊಂದುಲ್ಲೆ
ನಮಕ್ ಸೊ@@ ಲ@@ ಬೊ@@ ಗು ಅರ್ಥ ಆವ@@ ಂದಿನ ಒಂಜಿ ವಿ@@ ಜ್ಞ@@ ಾ@@ ನ ಇ@@ ತ್ತ್ಂಡ್
ಈ ಪ್ರಕರ@@ ಣ@@ ದ ಪತ್ತ@@ ೆಡ್ ದ@@ .@@ ಕ ಜಿಲ್ಲಾ ಪೊಲೀಸ್ ಅ@@ ಧ@@ ೀ@@ ಕ್ಷ@@ ಕ@@ ೆರಾಯಿನ ಶ್ರೀ ಋ@@ ಷ@@ ಿ@@ ಕೇ@@ ಶ್ ಸೋ@@ ನಾ@@ ವ@@ ಣೆ ಐ@@ .@@ ಪಿ@@ .@@ ಎ@@ ಸ್ ಮೆರ್ನ ನಿರ್@@ ದೇಶ@@ ನದ ಪ್ರಕಾರ ದ@@ .@@ ಕ ಜಿಲ್ಲಾ ಹೆ@@ ಚ್ಚ@@ ು@@ ವ@@ ರಿ ಪೊಲೀಸ್ ಅ@@ ಧ@@ ೀ@@ ಕ್ಷ@@ ಕ@@ ೆರಾಯಿನ ಶ್ರೀ ವಿ@@ .@@ ಬಿ ಭಾ@@ ಸ್@@ ಕ@@ ರ್ ಮೆರ್ನ ಮಾರ್ಗ@@ ದರ್ಶ@@ ನ@@ ಡ್ ಬಂಟ್@@ ವಾಳ ಉಪ@@ ವಿ@@ ಭಾ@@ ಗದ ಪೊಲೀಸ್ ಉ@@ ಪಾ@@ ಧ@@ ೀ@@ ಕ್ಷ@@ ಕ@@ ೆರಾಯಿನ ಶ್ರೀ ವೆ@@ ಲೈ@@ ಂಟ@@ ೈ@@ ನ್ ಡಿ@@ ಸೋ@@ ಜ@@ ಾ ಮೆರ್ನ ಮುತಾಲಿ@@ ಕೆ@@ ಡ್ ಬಂಟ್@@ ವಾಳ ನಗರ ಠ@@ ಾ@@ ಣ@@ ಾ ಪೊಲೀಸ್ ನಿ@@ ರೀಕ್ಷ@@ ಕ@@ ೆರಾಯಿನ ಚೆ@@ ಲು@@ ವ@@ ರಾಜ@@ ು ಬಿ , ಬಂಟ್@@ ವಾಳ ನಗರ ಠ@@ ಾ@@ ಣ@@ ಾ ಪೊಲೀಸ್ ಉಪ@@ ನಿ@@ ರೀಕ್ಷ@@ ಕೆರ್ ಬೊಕ್ಕ ಸಿ@@ ಬ್ಬ@@ ಂದಿ@@ ಲು ಬಂಟ್@@ ವಾಳ ಗ್ರಾ@@ ಮಾ@@ ಂತ@@ ರ ಪೊಲೀಸ್ ಠ@@ ಾ@@ ಣ@@ ಾ ಉಪ@@ ನಿ@@ ರೀಕ್ಷ@@ ಕೆರ್ ಅಂಚನೇ ಸಿ@@ ಬ್ಬ@@ ಂದಿ@@ ನಕುಲು ಬಂಟ್@@ ವಾಳ ನಗರ ಪೊಲೀಸ್ ಠ@@ ಾ@@ ಣ@@ ಾ ಅಪ@@ ರಾ@@ ಧ ವಿ@

In [21]:
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

/kaggle/lib/kagglegym:/kaggle/lib:/content/fairseq/


In [22]:
! python fairseq/fairseq_cli/preprocess.py \
    --joined-dictionary \
    --source-lang en --target-lang tu \
    --trainpref train --validpref valid --testpref valid \
    --destdir data-bin/trial --thresholdtgt 0 --thresholdsrc 0 \
    --workers 20

2024-07-04 08:36:20.921730: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 08:36:20.921837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 08:36:21.047793: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/kaggle/working/fairseq/fairseq/tasks/multires_hubert_pretraining.py:154: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  dictionaries = [ (Dictionary.load(f"{label_dir}/dict.{label}.txt") if label is not "" else None ) for label in self.cfg.labels]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.f

In [23]:
ls data-bin/trial

dict.en.txt        test.en-tu.en.idx   train.en-tu.en.idx  valid.en-tu.en.idx
dict.tu.txt        test.en-tu.tu.bin   train.en-tu.tu.bin  valid.en-tu.tu.bin
preprocess.log     test.en-tu.tu.idx   train.en-tu.tu.idx  valid.en-tu.tu.idx
test.en-tu.en.bin  train.en-tu.en.bin  valid.en-tu.en.bin


In [24]:
! python fairseq/fairseq_cli/train.py --fp16 \
    data-bin/trial \
    --source-lang en --target-lang tu \
    --arch transformer_iwslt_de_en --share-all-embeddings \
    --encoder-embed-dim	128 --encoder-ffn-embed-dim	128 \
    --encoder-layers	4 --encoder-attention-heads	4 \
    --decoder-embed-dim	128 --decoder-ffn-embed-dim	128 \
    --decoder-layers	4 --decoder-attention-heads	4 \
    --dropout 0.3 --weight-decay 0.0 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 0.01 --lr-scheduler inverse_sqrt --warmup-updates 10 \
    --max-tokens 4096 --update-freq 16 \
    --max-update 3000 \
    --keep-last-epochs	10 \
    --save-dir trained_models

2024-07-04 08:38:59.960244: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 08:38:59.960314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 08:38:59.962007: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
epoch 001:   0%|                                          | 0/4 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/n

In [25]:
ls trained_models

checkpoint743.pt  checkpoint746.pt  checkpoint749.pt  checkpoint752.pt
checkpoint744.pt  checkpoint747.pt  checkpoint750.pt  checkpoint_best.pt
checkpoint745.pt  checkpoint748.pt  checkpoint751.pt  checkpoint_last.pt


In [26]:
! python fairseq/fairseq_cli/interactive.py data-bin/trial \
  -s en -t tu \
  --distributed-world-size 1 \
  --path trained_models/checkpoint_best.pt \
  --batch-size 64 --buffer-size 2500 --beam 10 \
  --skip-invalid-size-inputs-valid-test \
  --input valid.en > valid.tu.translated

2024-07-04 09:16:22.991844: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 09:16:22.991916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 09:16:22.993765: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [31]:
from sacrebleu import corpus_bleu

# Step 1: Read the translated output
translated_output_file = "valid.tu.translated" 
translated_output = []
with open(translated_output_file, "r") as f:
    for line in f:
        translated_output.append(line.strip())

# Step 2: Read the reference translations
reference_file = "valid.tu"  
reference_translations = []
with open(reference_file, "r") as f:
    for line in f:
        reference_translations.append(line.strip())

# Step 3: Tokenize the translated output and reference translations
tokenized_output = [tokenizer.tokenize(sentence).lower() for sentence in translated_output]
tokenized_references = [[tokenizer.tokenize(sentence).lower()] for sentence in reference_translations]

# Step 4: Compute the BLEU score
bleu_score = corpus_bleu(tokenized_output, tokenized_references)
print("BLEU score:", bleu_score.score)


BLEU score: 10.600313379512592
